In [1]:
import torch as t
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from torch.utils import data
import os
import shutil

import visdom

input_dim = 10
n_hidden_1 = 32
n_hidden_2 = 16

n_hidden_3 = 8

output_dim = 2


vis = visdom.Visdom(env = u'face_angle')

In [2]:
#网络model
class Batch_Net1(nn.Module):
    def __init__(self,input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim):
        super().__init__()
        self.layer1 = nn.Sequential(
                                    nn.Linear(input_dim,n_hidden_1),
                                    nn.BatchNorm1d(n_hidden_1),
                                    nn.ReLU(True)
                                    )
        self.layer2 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_2),
                                    nn.BatchNorm1d(n_hidden_2),
                                    nn.ReLU(True)
                                    )
        self.layer3 = nn.Sequential(
                                    nn.Linear(n_hidden_2,n_hidden_3),
                                    nn.BatchNorm1d(n_hidden_3),
                                    nn.ReLU(True)
                                    )
        self.layer_out = nn.Sequential(
                                    nn.Linear(n_hidden_3,output_dim),
                                    #nn.BatchNorm1d(output_dim),
                                    #nn.ReLU(True)
                                       )
        #self.softmax = nn.Softmax()
 
        
        #x = self.layer4(x)      #想去掉某层，在这里就足够了。
        '''
        x1 = self.layer_out1(x)
        x2 = self.layer_out2(x)
        x = t.cat((x1,x2),1)           #这3行，即使和之前output_dim = 2效果是一样的，但是，说明可以使用cat。哈哈哈哈！
      
        x_out1 = self.layer_out1(x31)
        x_out2 = self.layer_out2(x32)
        x = t.cat((x_out1,x_out2),1)
        '''
    def forward(self,x):
        #print("x.shape",x.shape)
        x = self.layer1(x)
        #print("x.shape1",x.shape)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer_out(x)
        #x = self.softmax(x)
        #print(x)
        return x

In [3]:
#dataset类
from torch.utils import data
import os
import numpy as np


class Feature_Points(data.Dataset):
    def __init__(self,root):
        all_folds = os.listdir(root)
        folds = [os.path.join(root,img) for img in all_folds]
        self.all_files = []
        for path in folds:
            one_files = os.listdir(path)
            files = [os.path.join(path,img) for img in one_files]
            self.all_files.extend(files) 
         
        
    def __getitem__(self,index):
        txt_path = self.all_files[index]
        label = 1 if 'front_face' in txt_path.split("\\")[-2] else 0
        fo = open(txt_path)
        data_str = fo.read()
        array = np.array(data_str.split(",")).astype(float)
        fo.close()
        data = t.from_numpy(array)
        
        return data, label
    
    def __len__(self):
        return len(self.all_files)        
  

In [3]:
model = Batch_Net1(input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim)

In [4]:
model.load_state_dict(t.load("C:\\Users\\SiChen\\face_sys\\FC_network\\model_save\\fc_network_800_32_16_NoSoftmax_lr1e-4.pth"))

In [12]:

img_folder = "E:\\D_Continue\\newfolder601\\zhangwei"
txt_folder = "E:\\D_Continue\\newfolder202"
goal_front_face_folder = "E:\\D_Continue\\newfolder902\\clear_front_face"
goal_side_face_folder = "E:\\D_Continue\\newfolder902\\clear_side_face"


if os.path.exists(img_folder):
    print("img_folder文件夹存在")
else:
    print("img_folder文件夹不存在")
if os.path.exists(txt_folder):
    print("txt_folder文件夹存在")
else:
    print("txt_folder文件夹不存在")
if os.path.exists(goal_front_face_folder):
    print("goal_front_face_folder文件夹存在")
else:
    print("goal_front_face_folder文件夹不存在")
if os.path.exists(goal_side_face_folder):
    print("goal_side_face_folder文件夹存在")
else:
    print("goal_side_face_folder文件夹不存在")


model.cuda()
model.eval()

all_imgs = os.listdir(img_folder)
#all_imgs_path = [os.path.join(root,img) for img in all_folds]

for imgs_name in all_imgs:
    name = imgs_name.split(".")[0]
    txt_path = txt_folder+"\\"+name+".txt"
    img_path = img_folder+"\\"+imgs_name
    fo = open(txt_path)
    data_str = fo.read()
    array = np.array(data_str.split(",")).astype(float)
    fo.close()
    data_tensor = t.from_numpy(array)
    data_tensor = data_tensor.unsqueeze(0)                   #这行比较重要！！！！！！！！！！！！！！ 
    data_variable = Variable(data_tensor.float(),volatile=True).cuda()
    out = model(data_variable)
    _,pred = t.max(out.cpu(),1)
    
    if pred.data[0]==1:                                    #1表示正面人脸，0表示侧面人脸。
        shutil.copy(img_path,goal_front_face_folder)
    else:
        shutil.copy(img_path,goal_side_face_folder)

img_folder文件夹存在
txt_folder文件夹存在
goal_front_face_folder文件夹存在
goal_side_face_folder文件夹存在


In [4]:
import os
img_folder = "D:\\newfolder41_120_clear\\clear_face"
all_imgs = os.listdir(img_folder)
all_imgs[9].split(".")[0]

'picture_10022_'